In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a6d05c136601b8aa54b40b2c91fc9cadc67c9813a3bd1a3ef7ee4e501a7aad0a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession \
    .builder \
    .appName("ML_test") \
    .getOrCreate()

# Create Spark Context

In [ ]:
sc = spark.sparkContext


sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from google.colab import files


uploaded = files.upload()


Saving daily_weather.csv to daily_weather.csv


In [16]:
file='daily_weather.csv'

df = sqlContext.read.load(file,
                          format='com.databricks.spark.csv',
                          header='true',inferSchema='true')

#  Regression

In [24]:
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml.feature import VectorAssembler , StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline


In [25]:
df = df.na.drop()

In [26]:
featureColumns =df.columns[1:-2]

In [27]:
(trainingData, testData) = df.randomSplit([0.8,0.2], seed = 13234 )

In [33]:
df=df.withColumnRenamed("relative_humidity_3pm","label")

assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
scaler= StandardScaler(inputCol= "features", outputCol= "scaled_features")
lr= LinearRegression(featuresCol= "scaled_features")

In [34]:
pipeline = Pipeline(stages=[assembler,scaler, lr])

In [35]:
model = pipeline.fit(trainingData)

In [36]:
prediction = model.transform(testData)

In [37]:
prediction.show()

+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+------------------+--------------------+--------------------+------------------+
|number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|             label|            features|     scaled_features|        prediction|
+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+------------------+--------------------+--------------------+------------------+
|     9|915.0100000000116|  47.4979999999998|    163.10000000000005| 4.943637399999545|    195.89999999999992| 6.576603599999951|                  0.0|              0.0|     88.6000000

In [45]:
prediction.show(truncate=False)

+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|number|air_pressure_9am |air_temp_9am      |avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|label             |features                                                                                                                                              |scaled_features                                                                                        

In [55]:
from pyspark.sql import functions as func

prediction = prediction.withColumn('label2', func.round(prediction['label'], 2))
prediction=prediction.withColumn('prediction2', func.round(prediction['prediction'], 2))

In [56]:
prediction.select('label2','prediction2').show()

+------+-----------+
|label2|prediction2|
+------+-----------+
| 68.05|      65.58|
| 45.87|       37.4|
| 14.65|      24.87|
| 17.28|      23.03|
| 47.03|      49.85|
| 90.99|      63.44|
| 88.35|      63.74|
| 15.85|      35.18|
|  49.5|      51.18|
| 69.19|      43.13|
| 17.89|      13.25|
| 21.99|      21.95|
| 19.14|       20.9|
| 39.19|      40.76|
| 40.47|      42.23|
| 56.96|      52.56|
| 28.39|      43.81|
| 52.05|      34.89|
| 23.41|      23.94|
| 19.94|       25.3|
+------+-----------+
only showing top 20 rows



In [57]:
sc.stop()